In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [1]:
INPUT = "../subworkflows/pypsa-earth-sec/data/energy_totals_DF_2030.csv"

In [4]:
dict_cols = {"electricity rail":"rail electricity", "electricity residential":"residential electricity", 
                                   "domestic aviation":"aviation domestic", "domestic navigation":"navigation domestic", 
                                   "international navigation":"navigation international", "international aviation": "aviation international",
                                   "total rail":"rail oil", "total road":"road oil"}

energy_totals_2030 = pd.read_csv(INPUT, index_col=0)

In [7]:

energy_totals_2030.dropna(axis=0, inplace=True)
energy_totals_2030=energy_totals_2030.rename(columns=dict_cols)
energy_totals_2030=energy_totals_2030.sort_index(axis=1)

energy_totals_2030["road electricity"] =energy_totals_2030["road oil"] *0.14 # better fetched from network
energy_totals_2030["road hydrogen"] = energy_totals_2030["road oil"] *0.04 #better fetched from network
energy_totals_2030["road oil"] = energy_totals_2030["road oil"]  - energy_totals_2030["road electricity"] -energy_totals_2030["road hydrogen"] 
energy_totals_2030["aviation oil"] = energy_totals_2030["total international aviation"]  + energy_totals_2030["total domestic aviation"]
energy_totals_2030["navigation oil"] = energy_totals_2030["total international navigation"]  + energy_totals_2030["total domestic navigation"]
energy_totals_2030["navigation hydrogen"] = energy_totals_2030["navigation oil"] * 0.18 # better fetched from network
energy_totals_2030["navigation oil"]  = energy_totals_2030["navigation oil"]  - energy_totals_2030["navigation hydrogen"]


energy_totals_2030.drop(["total domestic aviation", "total international aviation", "total domestic navigation", "total international navigation"], axis=1, inplace=True)
energy_totals_2030["residential electricity"] = energy_totals_2030["residential electricity"] + energy_totals_2030["electricity residential space"]+ energy_totals_2030["electricity residential water"]
energy_totals_2030["services electricity"] = energy_totals_2030["services electricity"] + energy_totals_2030["electricity services space"]+ energy_totals_2030["electricity services water"]
energy_totals_2030 = energy_totals_2030.loc[:, (energy_totals_2030 != 0).any(axis=0)]
energy_totals_2030.columns

# I created energy_totals_base, 2040 and 2050 in the same way as above

#stacked_df=pd.concat([pd.DataFrame(base_energy_totals.sum()).T,pd.DataFrame(energy_totals_2030.sum()).T,pd.DataFrame(energy_totals_2040.sum()).T,pd.DataFrame(energy_totals_2050.sum()).T]).fillna(0)#.sort_index(axis=1)
stacked_df=pd.DataFrame(energy_totals_2030.sum()).T.fillna(0)#.sort_index(axis=1)
#stacked_df["residential gas"] = stacked_df["residential gas"] + stacked_df["residential heat gas"]
stacked_df["residential oil"] = stacked_df["residential oil"] + stacked_df["residential heat oil"]
stacked_df["residential biomass"] = stacked_df["residential biomass"] + stacked_df["residential heat biomass"]

stacked_df.drop(["residential heat oil", "residential heat biomass", "electricity residential space",
                 "electricity residential water", "total residential space", "total residential water" ], axis=1, inplace=True) #"total services space", "total services water", "residential heat gas"

stacked_df = stacked_df.loc[:, (stacked_df != 0).any(axis=0)]

new_cols = pd.Series(stacked_df.columns.to_list())
new_cols = new_cols.apply(lambda x: x.split(" ")[1] + " " +x.split(" ")[0])
stacked_df.columns = new_cols

stacked_df = stacked_df.sort_index(axis=1)

fig = plt.figure(figsize=(9, 4))
fig.tight_layout()
ax = plt.subplot(111)
to_plot= stacked_df.sort_index()#axis=1)
to_plot.index = ["EG_2030"]
to_plot.plot.bar(stacked=True, ax=ax)
plt.xticks(rotation=0)


l3 = plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0, prop={'size': 8.19})

plt.ylabel("Final energy demand [TWh]")
plt.grid()

plt.grid(color='grey', linestyle='--', linewidth=0.5, zorder=0)
plt.savefig("OCEANIA_demand_proj_n.pdf", bbox_inches='tight')
plt.show()


KeyError: 'total international aviation'